In [2]:
import numpy as np
import pickle
from ipywidgets import widgets
import anvil.server

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
# load model
model_sent = load_model('sentiment_analysis.h5')

In [4]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [6]:
anvil.server.connect("UFYDTFT7QXDA3TXGYCEH6TKZ-IWRSVDXUFFPCWVFX")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [11]:
@anvil.server.callable
def predict_review(new_sentences, model=model_sent, max_length=100, show_padded_sequence=False):
    
    result_dict = dict()
    result_dict['Input'] = new_sentences
    
    if isinstance(new_sentences, str):
        new_sentences = [' '.join(new_sentences.split(" "))]
    
    trunc_type='post' 
    padding_type='post'

    # text to sequence
    new_text_sequences = tokenizer.texts_to_sequences(new_sentences)

    # Pad all sequences for the new reviews
    new_reviews_padded = pad_sequences(new_text_sequences, maxlen=max_length, 
                                     padding=padding_type, truncating=trunc_type)             

    classes = model.predict(new_reviews_padded)

    # The closer the class is to 1, the more positive the review is
    for x in range(len(new_sentences)):
        
        # Print its predicted class
        if classes[x] > 0.5:
            result_dict['Result'] = "Positive"
            #print("Positive")
        else:
            result_dict['Result'] = "Negative"
            #print("Negative")
        result_dict['Probability'] = classes[x]
    
    return result_dict['Input'], result_dict['Result'], result_dict['Probability']